In [1]:
import angr
from angrutils import *

In [500]:
# binary = '/data2/zhangzheng1/data/example-c/build/src/test_string'
# binary = './test'
binary = '/usr/bin/base64'

p = angr.Project(binary, auto_load_libs=False)

In [501]:
# main = p.loader.main_object.get_symbol("main")
# print(hex(main.rebased_addr))
# print(hex(p.entry))

In [502]:
# cfg = p.analyses.CFGEmulated(fail_fast=True, starts=[main.rebased_addr])
cfg = p.analyses.CFGFast()

WARNING | 2020-07-02 10:30:09,232 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00a0 with 8 unconstrained bytes referenced from 0x4031b1 (PLT.__gmon_start__+0x1a81 in base64 (0x4031b1))
WARNING | 2020-07-02 10:30:09,235 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00a8 with 8 unconstrained bytes referenced from 0x4031b2 (PLT.__gmon_start__+0x1a82 in base64 (0x4031b2))
WARNING | 2020-07-02 10:30:09,238 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00b0 with 8 unconstrained bytes referenced from 0x4031b3 (PLT.__gmon_start__+0x1a83 in base64 (0x4031b3))
WARNING | 2020-07-02 10:30:09,241 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00b8 with 8 unconstrained bytes referenced from 0x4031b5 (PLT.__gmon_start__+0x1a85 in base64 (0x4031b5))
WARNING | 2020-07-02 10:30:09,245 | angr.state_plugins.symbolic_memory | Filling memory at 0x7ffffffffff00c0 with 8 unconstrained bytes referenced from 0x4031b7

In [503]:
# cfg.kb.functions.get('main').get_call_sites()

In [504]:
# for b in sorted(cfg.kb.functions.get('main').blocks, key=lambda x: x.addr):
#     b.pp()
#     print()

In [505]:
# [b for b in cfg.kb.functions.get('__libc_start_main').blocks][0].pp()

In [506]:
# cfg.kb.functions.get('main')

In [507]:
def checkNop(blk): # 检查blk全部为nop的情况
    try:
        if not blk: return True
        if blk.size == 0: return True
        return all([blk.capstone.insns[i].insn.insn_name() == 'nop' for i in range(blk.instructions)])
    except Exception as e:
        print('[NOP]{} {}'.format(blk, blk.size))
        return False

def getcov(func, s):

    blkList = list(func.blocks)

    for blk in blkList: # 遍历该函数所有block
        if blk.addr not in s:
            s.add(blk.addr)
            
            node = cfg.get_node(blk.addr) # 将block转换为node

            if node.has_return: # 该block包含ret则不遍历successors
                s.add(node.addr)
            elif checkNop(blk): # 该block全部为nop指令
                s.add(node.addr)
            else:
                for ns in node.successors: # 遍历该block的后续block
                    if ns.addr not in s:
                        if checkNop(ns.block): # 该block全部为nop指令
                            s.add(ns.addr)
                        elif ns.has_return: # 该block包含ret
                            s.add(ns.addr)
                        else:
                            if ns.addr in func.block_addrs: # func内条件跳转（angr解析func得到的blk会少一部份）
                                continue
                            else:
                                try:
                                    new_func = cfg.kb.functions.get_by_addr(ns.addr) # jmp或call调用其它函数
                                    getcov(new_func, s) # 递归解析下一个函数
                                except KeyError as e: # func内跳转到angr未解析为该func的blk
                                    blkList.append(ns.block)
#                                     print('[!] {} {} {}'.format(func.name, hex(blk.addr), hex(ns.addr)))
                                    continue
    
    return len(s)

In [519]:
func2cov = {}

for addr, func in cfg.kb.functions.items():
    if func.name not in func2cov:
        func2cov[func.name] = getcov(func, set())
    else:
        func2cov[func.name] = max(func2cov[func.name], getcov(func, set()))

# main_cov = getcov(cfg.kb.functions.get('main'), set())
# all_blk = len(cfg.graph.nodes())
# print('[+] {:.2%}'.format(main_cov/all_blk))

In [511]:
# sub_403162 0x40318f 0x403b9f
# func = cfg.functions.get('sub_402e10')
# # print(0x402e10 <= 0x403a10 < 0x40343f)
# for b in sorted(func.blocks, key=lambda x: x.addr):
# # for b in func.blocks:
# #     if b.addr == 0x40336a:
#     b.pp()
#     print()
# print(hex(func.addr))
# print(hex(func.addr+func.size))
# print(func.size)

0x402e10:	push	r15
0x402e12:	push	r14
0x402e14:	mov	r15d, r8d
0x402e17:	push	r13
0x402e19:	push	r12
0x402e1b:	mov	r14, rdi
0x402e1e:	push	rbp
0x402e1f:	push	rbx
0x402e20:	mov	ebp, r9d
0x402e23:	mov	r13, rcx
0x402e26:	and	ebp, 2
0x402e29:	sub	rsp, 0xa8
0x402e30:	mov	rax, qword ptr [rsp + 0xe0]
0x402e38:	mov	qword ptr [rsp], rsi
0x402e3c:	mov	qword ptr [rsp + 0x18], rdx
0x402e41:	mov	dword ptr [rsp + 0x38], r9d
0x402e46:	mov	qword ptr [rsp + 0x28], rax
0x402e4b:	mov	rax, qword ptr [rsp + 0xe8]
0x402e53:	mov	qword ptr [rsp + 0x58], rax
0x402e58:	mov	rax, qword ptr [rsp + 0xf0]
0x402e60:	mov	qword ptr [rsp + 0x50], rax
0x402e65:	mov	rax, qword ptr fs:[0x28]
0x402e6e:	mov	qword ptr [rsp + 0x98], rax
0x402e76:	xor	eax, eax
0x402e78:	call	0x401460

0x402e7d:	cmp	r15d, 0xa
0x402e81:	mov	qword ptr [rsp + 0x60], rax
0x402e86:	ja	0x403a6e

0x402e8c:	mov	eax, r15d
0x402e8f:	mov	r10, qword ptr [rsp]
0x402e93:	jmp	qword ptr [rax*8 + 0x406300]

0x402e9e:	mov	byte ptr [rsp + 0x17], 1
0x402ea3:	mov	qwo

In [514]:
# 0x403162 in func.block_addrs

False

In [520]:
# cfg.get_node(0x403162).block.pp()

In [521]:
main = cfg.functions.get('main')
blkcnt = 0
for blk in main.blocks:
    blkcnt += 1
    for s in cfg.get_node(blk.addr).successors:
        if not (main.addr <= s.addr <= main.addr + main.size):
            name = cfg.functions.get_by_addr(s.addr).name
            print('{} {}'.format(name, func2cov[name]))
print('main: {} {}'.format(blkcnt, func2cov['main']))

sub_402c10 20
setlocale 2
bindtextdomain 2
textdomain 2
sub_405a30 5
getopt_long 2
sub_4044f0 331
dcgettext 2
fopen 2
error 2
sub_4049b0 63
sub_4020a0 63
exit 2
sub_4020a0 63
dcgettext 2
sub_402be0 8
sub_4042d0 342
sub_404e30 403
__errno_location 2
sub_401d70 98
sub_401ae0 135
error 2
sub_405400 46
sub_4042d0 342
dcgettext 2
__errno_location 2
__errno_location 2
error 2
error 2
main: 55 878


In [522]:
for f in sorted(func2cov, key=lambda x: func2cov[x], reverse=True)[:40]:
    print('{}: {}'.format(f, func2cov[f]))

main: 878
sub_40366a: 468
sub_402b20: 409
sub_404e30: 403
sub_404d20: 401
sub_4040e2: 344
sub_404143: 344
sub_4041b0: 344
sub_4041c0: 344
sub_4040f7: 342
sub_40415c: 342
sub_4042d0: 342
sub_404450: 338
sub_404460: 338
sub_404480: 338
sub_404390: 337
sub_4041cc: 335
sub_404290: 332
sub_4042a0: 332
sub_4042c0: 332
sub_404080: 331
sub_4040a0: 331
sub_4040b0: 331
sub_4040d0: 331
sub_4044a0: 331
sub_4044b0: 331
sub_4044d0: 331
sub_4044f0: 331
sub_403bb0: 330
sub_403ed2: 304
sub_403fd0: 304
sub_403efb: 302
sub_4037bb: 284
sub_403e60: 284
sub_403771: 282
sub_402e10: 279
sub_402e9a: 279
sub_403162: 279
sub_402d40: 174
sub_4055a0: 156


In [113]:
# plot_cfg(cfg, "test", asminst=True, remove_imports=True, remove_path_terminator=True)